In [ ]:
# Install dependencies
%pip install yfinance -q

# Imports
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from kaggle_secrets import UserSecretsClient
import yfinance as yf # Yahoo Finance API
import certifi # SSL cert
from time import sleep
import plotly.express as px # Plotting library
import random

# Util functions
def remove_empty_columns_from_right(df):
    df = df.copy()
    while len(df.columns) > 0 and not df[df.columns[-1]].any():
        df.drop(df.columns[-1], axis=1, inplace=True)
    return df

def reset_index(df):
    df = df.copy()
    df.index = list(range(len(df.index)))
    return df

In [ ]:
CURRENCY_SIGN = '$'
N_DIGITS_AFTER_POINT = 2

# Parse Interactive Brokers reports unless trades or positions are provided

In [ ]:
trades = None
positions = None

input_filenames = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        if filename == 'trades.csv':
            trades = pd.read_csv(path)
            continue
        if filename == 'positions.csv':
            positions = pd.read_csv(path)
            continue
        input_filenames.append(path)

if trades is None and positions is None:
    print('Parse trades from Interactive Brokers reports')
elif positions is None:
    print('Trades are provided')

In [ ]:

if trades is None and positions is None:
    print('Parse Interactive Brokers reports')
    # Concatenate annual reports into a single one
    interactive_brokers_reports = []

    sorted_filenames = sorted(input_filenames)
    for report_path in sorted_filenames:
        interactive_brokers_reports.append(pd.read_csv(report_path, header=None, names=list(range(100))))

    full_interactive_brokers_report = pd.concat(interactive_brokers_reports, ignore_index=True)

    cut_report = remove_empty_columns_from_right(full_interactive_brokers_report)
    cut_report.shape

    trades_table = remove_empty_columns_from_right(cut_report[cut_report[0] == 'Trades'])
    trades_table.drop(0, axis=1, inplace=True)
    trades_table.columns = trades_table.iloc[0].values
    trades_table.drop(trades_table.index[0], axis=0, inplace=True)
    trades_table = reset_index(trades_table)
    trades_table.shape

    trades = trades_table[(trades_table['Header'] == 'Data') & (trades_table['Asset Category'] == 'Stocks')].copy()
    trades.drop(['Header', 'Asset Category'], axis=1, inplace=True)
    trades['Date/Time'] = trades['Date/Time'].apply(lambda s: pd.to_datetime(s))
    trades.sort_values(by='Date/Time', inplace=True)
    trades = reset_index(trades)
    trades = trades[['Symbol', 'Quantity', 'T. Price']]
    trades = pd.DataFrame({
        'symbol': trades['Symbol'],
        'quantity': trades['Quantity'],
        'price': trades['T. Price']
    })
    trades.to_csv('trades.csv', index=False)

# Calculate positions from trades unless positions are provided

In [ ]:
if positions is None:
    print('Calculate positions from trades')
else:
    print('Positions are provided')

In [ ]:
if positions is None:
    print('Calculate positions from trades')
    symbol_state = {}
    for index, trade in trades.iterrows():
        symbol = trade['symbol']
        symbol_state.setdefault(symbol, {'position': []})
        quantity = int(trade['quantity'])
        price = float(trade['price'])
        if quantity > 0:
            symbol_state[symbol]['position'].append({'quantity': quantity, 'price': price})
        else:
            position = symbol_state[symbol]['position']
            quantity = -quantity
            while quantity > 0:
                if position[0]['quantity'] <= quantity:
                    quantity -= position[0]['quantity']
                    position.pop(0)
                else:
                    position[0]['quantity'] -= quantity
                    quantity = 0

    try:
        # Merger. APHA & TLRY
        if UserSecretsClient().get_secret('use-hardcoded-mergers') == 'True':
            symbol_state['APHA'] = {'position': []}
            symbol_state['TLRY'] = {'position': [{'quantity': 2.5143, 'price': 5.66}]}
    except Exception as exc_info:
        print(str(exc_info))

    positions = pd.DataFrame()
    positions['symbol'] = symbol_state.keys()
    positions.index = symbol_state.keys()
    positions['quantity'] = None
    positions['average_price'] = None
    for symbol, state in symbol_state.items():
        quantity = 0
        cost = 0.0
        for item in state['position']:
            quantity += item['quantity']
            cost += item['quantity'] * item['price']
        positions.at[symbol, 'quantity'] = quantity
        positions.at[symbol, 'average_price'] = cost / quantity if quantity > 0 else 0.0
    positions = positions[positions['quantity'] != 0].copy()
    positions.sort_values(by='symbol', inplace=True)
    positions = reset_index(positions)

    positions.to_csv('positions.csv', index=False)

# Collect information about current price, sector and industry

In [ ]:
# Suppress SSL warning
os.environ['SSL_CERT_FILE'] = certifi.where()

class SecurityProfile:

    def __init__(self, ticker):
        self.ticker = ticker
        
    def data(self):
        yf_ticker = yf.Ticker(self.ticker)
        info = yf_ticker.info
        history = yf_ticker.history(period='1d')
        return {
            'name': info['longName'],
            'price': history['Close'][-1],
            'sector': info.get('sector', 'N/A'),
            'industry': info.get('industry', 'N/A')
        }

class FakeSecurityProfile:

    def __init__(self, ticker):
        self.ticker = ticker
        
    def data(self):
        industry_id = random.randint(0, 20)
        return {
            'name': self.ticker,
            'price': random.randint(5, 15),
            'sector': 'N/A' if industry_id < 4 else f'Sector {int(industry_id / 4)}',
            'industry': 'N/A' if industry_id < 4 else f'Idustry {industry_id}'
        }

class SecurityProfileCollection:
    
    def __init__(self, tickers, fake=False):
        self.tickers = tickers
        self.fake = fake
        
    def dataframe(self):
        result = pd.DataFrame(columns=['symbol', 'sector', 'industry', 'name', 'price'])
        success = False
        for i in range(500):
            if success:
                break
            print(f'!!! {i} !!!')
            finished = True
            try:
                for index, ticker in enumerate(self.tickers):
                    if index in result.index and result.at[index, 'sector'] is not None:
                        continue
                    print(index, ticker)
                    security_profile = FakeSecurityProfile(ticker) if self.fake else SecurityProfile(ticker)
                    data = security_profile.data()
                    result.at[index, 'symbol'] = ticker
                    result.at[index, 'sector'] = data['sector']
                    result.at[index, 'industry'] = data['industry']
                    result.at[index, 'name'] = data['name']
                    result.at[index, 'price'] = data['price']
            except Exception as ex:
                print(ex)
                raise
                finished = False
            if not finished:
                sleep(10)
            else:
                success = True
        return result

portfolio = positions.copy()
portfolio = portfolio[['symbol', 'quantity', 'average_price']]

portfolio.index = list(range(portfolio.shape[0]))

portfolio = portfolio.merge(SecurityProfileCollection(portfolio['symbol'], fake=False).dataframe(), on='symbol')

In [ ]:
print(f'Cost: {(portfolio["average_price"] * portfolio["quantity"]).sum():.2f}{CURRENCY_SIGN}')
print(f'Value: {(portfolio["quantity"] * portfolio["price"]).sum():.2f}{CURRENCY_SIGN}')
portfolio

# Diversification by sectors and industries

In [ ]:
portfolio['value'] = (portfolio['quantity'] * portfolio['price']).apply(lambda x: round(x, N_DIGITS_AFTER_POINT))
shares = portfolio[portfolio['sector'] != 'N/A'].copy()
shares['total'] = 'Shares'
fig = px.sunburst(
    shares,
    path=['total', 'sector', 'industry', 'symbol'],
    values='value',
    width=750,
    height=750
)
fig.show()
funds = portfolio[portfolio['sector'] == 'N/A'].copy()
funds['total'] = 'Funds'
fig = px.sunburst(
    funds,
    path=['total', 'symbol'],
    values='value',
    width=750,
    height=750
)
fig.show()

# Portfolio specific analytics

In [ ]:
def sum_by_symbols(symbols):
    pd.DataFrame({'symbol': portfolio['symbol'], 'amount': portfolio['quantity'] * portfolio['price']})
    sub_df = portfolio[[symbol in symbols for symbol in portfolio['symbol']]]
    return (sub_df['quantity'] * sub_df['price']).sum()

shares_symbols = set(portfolio[portfolio['sector'] != 'N/A']['symbol'])
shares_symbols.update({'VT', 'VXF', 'VXUS'})
shares_sum = sum_by_symbols(shares_symbols)
gold_symbols = {'GLD'}
gold_sum = sum_by_symbols(gold_symbols)
material_symbols = {'SLV', 'UNG', 'WEAT'}
material_sum = sum_by_symbols(material_symbols)
obligation_symbols = {'TLT', 'TIP', 'BNDW'}
obligation_sum = sum_by_symbols(obligation_symbols)
rest = set(portfolio['symbol']) - shares_symbols - gold_symbols - material_symbols - obligation_symbols
if len(rest) != 0:
    raise Exception(f'Unprocessed symbols: {rest}')
total = sum((shares_sum, gold_sum, material_sum, obligation_sum))
print(f'Shares: {shares_sum / total * 100:.02f}%')
print(f'Obligations: {obligation_sum / total * 100:.02f}%')
print(f'Gold: {gold_sum / total * 100:.02f}%')
print(f'Materials: {material_sum / total * 100:.02f}%')